In [4]:
import time

num_list = [1]*8000000
sum_num = 0

start_time = time.time()
for i in num_list:
    sum_num += i
print(f"Used Time:{time.time()- start_time}")

Used Time:0.8052911758422852


In [3]:
import time

num_list = [1]*8000000
sum_num, n = 0, 8

def chunk_sum(num_list: list):
    return sum(num_list)

start_time = time.time()
chunk_size = len(num_list) // n
chunks = [num_list[i * chunk_size:(i + 1) * chunk_size] for i in range(n)]
for _ in chunks:
    sum_num += chunk_sum(_)
print(f"Used Time:{time.time()- start_time}")

Used Time:0.0834810733795166


In [12]:
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

num_list = [1]*8000000
sum_num, n = 0, 4

def chunk_sum(num_list: list):
    return sum(num_list)

# 使用线程
start_time = time.time()
chunk_size = len(num_list) // n
chunks = [num_list[i * chunk_size:(i + 1) * chunk_size] for i in range(n)]
with ThreadPoolExecutor(max_workers= 2) as executor:
    futures = [executor.submit(sum, chunk) for chunk in chunks]  # 提交任务
    results = [future.result() for future in as_completed(futures)]  # 获取结果
sum_num  = sum(results)
print(f"Used Time:{time.time()- start_time}")

# 使用进程
# start_time = time.time()
# chunk_size = len(num_list) // n

# with ProcessPoolExecutor(max_workers=n) as executor:
#     futures = [
#         executor.submit(chunk_sum, i * chunk_size, (i + 1) * chunk_size, num_list)
#         for i in range(n)
#     ]
#     results = [f.result() for f in futures]
# sum_num  = sum(results)
# print(f"Used Time:{time.time()- start_time}")

Used Time:0.14263606071472168


In [18]:
import time
import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

num_list = [i for i in range(8000000)]
sum_num, n = 0, 4

def write_to_txt_one(num_list):
    start_time = time.time()
    with open('./tmp.txt', 'w', encoding= 'utf-8') as f:
        for _ in num_list:
            f.write(f"{_}")
    print(f"Used Time:{time.time()- start_time}")

def write_to_txt_thread(num_list, lock):
    start_time = time.time()
    with lock:
        with open('./tmp.txt', 'w', encoding= 'utf-8') as f:
            for _ in num_list:
                f.write(f"{_}")
    print(f"Used Time:{time.time()- start_time}")

def main_thread():
    lock = threading.Lock()
    threads = []
    for i in range(0, len(num_list), chunk_size):
        chunk = num_list[i:i + chunk_size]
        thread = threading.Thread(target=write_to_txt_thread, args=(chunk, lock))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

write_to_txt_one(num_list)
main_thread()

Used Time:1.6002354621887207
Used Time:0.3817405700683594
Used Time:0.7378873825073242
Used Time:1.090383529663086
Used Time:1.4325108528137207


In [ ]:
import threading
import time
lock = threading.Lock()

def llm_api_result(num):
    time.sleep(2)
    return str(num+1)

def write_to_text(num, lock):
    with lock:
        with open('./text.txt', 'a+', encoding= 'utf-8') as f:
            f.write(llm_api_result(num))

def main():
    threads = []
    for i in range(0, 10):
        thread = threading.Thread(target=write_to_text, args= (i, lock))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
main()

In [1]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading

def llm_api_result(num):
    time.sleep(2)
    return f"{num}"*10

def write_to_file(num):
    with open("output.txt", "a", encoding="utf-8") as f:
        content = f"Thread-{num}: " + llm_api_result(num)
        for char in content:
            f.write(char)
            time.sleep(0.001)
        f.write("\n")

def main():
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(write_to_file, num) for num in range(10)]
        for future in futures:
            future.result()

if __name__ == "__main__":
    start_time = time.time()
    main()
    print("Used Time:", time.time()- start_time)

Used Time: 6.367322683334351
